In [1]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Ścieżka do pliku chromedriver (zmień na właściwą lokalizację)
driver_path = "/Users/michal/Downloads/chromedriver-mac-x64/chromedriver"

# Inicjalizacja sterownika Chrome
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Ustawienie WebDriverWait
wait = WebDriverWait(driver, 30)

# Otwieramy stronę z forecastem MSFT
url = "https://www.tradingview.com/symbols/NASDAQ-MSFT/forecast/"
driver.get(url)

# Czekamy, aż pojawią się wszystkie znaczniki <script> z type="application/ld+json"
scripts = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, "//script[@type='application/ld+json']"))
)

data_list = []

# Iterujemy po znalezionych skryptach i próbujemy sparsować zawartość jako JSON
for script in scripts:
    try:
        json_text = script.get_attribute("innerHTML")
        data = json.loads(json_text)
        data_list.append(data)
    except Exception as e:
        print("Błąd parsowania JSON:", e)

# Wypisujemy wszystkie pobrane dane
print("Wszystkie dane JSON-LD:")
for entry in data_list:
    print(json.dumps(entry, indent=2, ensure_ascii=False))

# Przykład: wyszukujemy w danych FAQ informację o prognozowanej cenie
for entry in data_list:
    if isinstance(entry, dict) and entry.get("@type") == "FAQPage":
        main_entities = entry.get("mainEntity", [])
        for entity in main_entities:
            if entity.get("name") == "What is MSFT price target?":
                answer_text = entity.get("acceptedAnswer", {}).get("text", "")
                print("\nMSFT Price Target Info:")
                print(answer_text)

# Zamykamy sterownik
driver.quit()


Wszystkie dane JSON-LD:
{
  "@context": "http://schema.org",
  "@type": "BreadcrumbList",
  "itemListElement": [
    {
      "@type": "ListItem",
      "position": 1,
      "item": {
        "@id": "https://www.tradingview.com/markets/",
        "name": "Markets"
      }
    },
    {
      "@type": "ListItem",
      "position": 2,
      "item": {
        "@id": "https://www.tradingview.com/markets/usa/",
        "name": "USA"
      }
    },
    {
      "@type": "ListItem",
      "position": 3,
      "item": {
        "@id": "https://www.tradingview.com/markets/stocks-usa/market-movers-large-cap/",
        "name": "Stocks"
      }
    },
    {
      "@type": "ListItem",
      "position": 4,
      "item": {
        "@id": "https://www.tradingview.com/markets/stocks-usa/sectorandindustry-sector/technology-services/",
        "name": "Technology Services"
      }
    },
    {
      "@type": "ListItem",
      "position": 5,
      "item": {
        "@id": "https://www.tradingview.com/markets

In [200]:
import json
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Ścieżka do pliku chromedriver (zmień na właściwą lokalizację)
driver_path = "/Users/michal/Downloads/chromedriver-mac-x64/chromedriver"

# Inicjalizacja sterownika Chrome
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Ustawienie WebDriverWait na 30 sekund
wait = WebDriverWait(driver, 30)

# Ładujemy stronę główną dla AMD (bez /forecast/)
url_main = "https://www.tradingview.com/symbols/NASDAQ-AMD/"
driver.get(url_main)

# pobierabie Beta ze strony overview
beta_element = driver.find_element(
    By.XPATH,
    "//div[contains(@class, 'block-GgmpMpKr')]//div[contains(text(),'Beta (1Y)')]/following::div[contains(@class, 'value-GgmpMpKr')][1]"
)
beta = beta_element.text
print("Beta (1Y):", beta)

# pobierabie Shares float ze strony overview
value_element = driver.find_element(
    By.XPATH,
    "//div[contains(@class, 'apply-overflow-tooltip') and contains(@class, 'value-GgmpMpKr') and contains(text(), 'B')]"
)

shares_text = value_element.text
cleaned = shares_text.replace(u'\xa0', ' ')
cleaned = re.sub(r'[\u200E\u202A\u202C\u202F]', '', cleaned)
shares_text = cleaned
print("Shares cleaned:", shares_text)

# Ładujemy stronę technikaliów (technicals) dla AMD
url_technicals = "https://www.tradingview.com/symbols/NASDAQ-AMD/technicals/"
driver.get(url_technicals)
time.sleep(1)  # krótka przerwa na załadowanie strony

# Funkcja pomocnicza – konwertuje tekst zakładki na sufiks (np. "1 minute" -> "1m")
def convert_tab_text_to_suffix(tab_text):
    parts = tab_text.lower().split()
    mapping = {
        "minute": "m", "minutes": "m",
        "hour": "h", "hours": "h",
        "day": "d",
        "week": "w",
        "month": "M"
    }
    if len(parts) >= 2:
        number = parts[0]
        unit = parts[1]
        return number + mapping.get(unit, "")
    return tab_text

# Funkcja wyciągająca dane tabelaryczne z aktualnie wyświetlonej sekcji technikaliów
def extract_table_data():
    extracted_data = {}
    try:
        tables_wrapper = driver.find_element(By.CSS_SELECTOR, "div.tablesWrapper-kg4MJrFB.tabletVertical-kg4MJrFB")
        table_containers = tables_wrapper.find_elements(By.CSS_SELECTOR, "div.container-hvDpy38G")
    except Exception as e:
        print("Brak tabeli:", e)
        return {}
    
    for container in table_containers:
        try:
            title_element = container.find_element(By.CSS_SELECTOR, "div.title-hvDpy38G a.link-hvDpy38G")
            section_title = title_element.text.strip()
        except Exception as e:
            section_title = "Brak tytułu"
        try:
            table = container.find_element(By.CSS_SELECTOR, "table.table-hvDpy38G")
        except Exception as e:
            continue
        rows = table.find_elements(By.CSS_SELECTOR, "tr.row-hvDpy38G")
        rows_data = []
        for row in rows:
            cells = row.find_elements(By.CSS_SELECTOR, "td.cell-hvDpy38G")
            if len(cells) >= 3:
                name = cells[0].text.strip()
                value = cells[1].text.strip()
                action = cells[2].text.strip()
                rows_data.append({
                    "Name": name,
                    "Value": value,
                    "Action": action
                })
        extracted_data[section_title] = rows_data
    return extracted_data

# Pobieramy wszystkie zakładki z interwałami – zakładki są umieszczone w elemencie o id "technicals-intervals-tabs"
tabs_container = driver.find_element(By.ID, "technicals-intervals-tabs")
tab_buttons = tabs_container.find_elements(By.TAG_NAME, "button")

# Definiujemy porządek zakładek. Uwaga: Identyfikatory zakładek:
# dla 1 minute, 5 minutes, 15 minutes, 30 minutes, 1 hour, 2 hours, 4 hours, 1 day, 1 week, 1 month
order_mapping = {
    # "1m": 0,       # 1 minute (id="1m")
    # "5m": 1,
    # "15m": 2,
    # "30m": 3,
    # "1h": 4,
    # "2h": 5,
    # "4h": 6,
    "1D": 7,       # 1 day – zazwyczaj id="1D" (z dużą literą)
    "1W": 8,       # 1 week
    "1M": 9        # 1 month
}

# Sortujemy przyciski zakładek według ustalonego porządku (jeśli nie znajdziemy id, przypisujemy dużą wartość)
sorted_tabs = sorted(tab_buttons, key=lambda btn: order_mapping.get(btn.get_attribute("id"), 999))

final_flattened_data = {}

# Iterujemy po posortowanych zakładkach
for button in sorted_tabs:
    tab_id = button.get_attribute("id")
    # Pomijamy zakładki, które nie są interesujące (np. "More" lub bez id)
    if tab_id not in order_mapping:
        continue

    tab_text = button.text.strip()  # np. "1 minute", "5 minutes", "1 month", itd.
    suffix = convert_tab_text_to_suffix(tab_text)
    print(f"Przetwarzam zakładkę: {tab_id} - {tab_text} (sufiks: {suffix})")
    
    # Jeśli zakładka nie jest aktywna, klikamy ją
    if button.get_attribute("aria-selected") != "true":
        button.click()
        time.sleep(1)  # oczekujemy, aż dane się załadują
    
    # Pobieramy dane tabelaryczne z aktywnej zakładki
    tab_extracted_data = extract_table_data()
    
    # Spłaszczamy dane – dla każdego wiersza tworzymy nowe klucze ze sufiksem interwału (np. "Relative Strength Index (14) - Value 1m")
    for category, items in tab_extracted_data.items():
        for item in items:
            name = item.get("Name", "").strip()
            value = item.get("Value", "").strip()
            action = item.get("Action", "").strip()
            key_value = f"{name} - Value tv {suffix}"
            key_action = f"{name} - Action tv {suffix}"
            final_flattened_data[key_value] = value
            final_flattened_data[key_action] = action

    # Jeśli przetworzyliśmy zakładkę "1M" (1 month), kończymy iterację
    if tab_id == "1M":
        break

url_forecast = url_main + "forecast/"

driver.get(url_forecast)

# Dodatkowe oczekiwanie po załadowaniu strony
time.sleep(1)

# Czekamy na pojawienie się elementu z ceną (dla przykładu selektor CSS)
price_element = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "span.last-JWoJqCpY.js-symbol-last span"))
)
price_value = price_element.text 
print(price_value)


# Poczekaj aż pojawią się wszystkie skrypty JSON-LD
scripts = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, "//script[@type='application/ld+json']"))
)

data_list = []
for script in scripts:
    try:
        json_text = script.get_attribute("innerHTML")
        data = json.loads(json_text)
        data_list.append(data)
    except Exception as e:
        print("Błąd parsowania JSON:", e)
        

# Pobieramy dane sentymentu analityków z fragmentu HTML wykorzystując Selenium
# Zakładamy, że fragment zawierający sentyment ma klasę "wrap-GNeDL9vy"
sentiment_wrap = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.wrap-GNeDL9vy"))
)

# Pobieramy wszystkie elementy z tytułami i odpowiadającymi im wartościami
title_elements = sentiment_wrap.find_elements(By.CSS_SELECTOR, "div.title-GNeDL9vy")
value_elements = sentiment_wrap.find_elements(By.CSS_SELECTOR, "div.value-GNeDL9vy")

# Pobieramy tekst z elementów
titles = [el.text for el in title_elements]
values = [el.text for el in value_elements]

# Tworzymy słownik – łączymy tytuły z wartościami (konwertujemy wartości do int)
sentiment_data = {title: int(value) for title, value in zip(titles, values)}
sentiment_data = { f"{k} tv": v for k, v in sentiment_data.items() }
print("Sentiment data:", sentiment_data)

driver.quit()

# Przygotowujemy zmienne, które będziemy szukać w tekstach odpowiedzi
eps_last_quarter = None
eps_last_quarter_estimation = None
eps_next_quarter_expected = None
price_target = None
max_estimate = None
min_estimate = None
num_analysts = None
next_quarter_revenue = None


# Funkcja pomocnicza do próby wyszukania wzorca w danym tekście
def search_pattern(pattern, text):
    match = re.search(pattern, text, re.IGNORECASE)
    return match.groups() if match else None


# Przechodzimy przez wszystkie dane JSON-LD i szukamy FAQPage
for entry in data_list:
    if isinstance(entry, dict) and entry.get("@type") == "FAQPage":
        main_entities = entry.get("mainEntity", [])
        for entity in main_entities:
            question = entity.get("name", "")
            answer_text = entity.get("acceptedAnswer", {}).get("text", "")

            # Usuń ewentualne znaczniki HTML (proste podejście)
            answer_text_clean = re.sub(r'<[^>]+>', '', answer_text)

            # Parsowanie EPS
            if "How has AMD EPS performed lately" in question:
                # Przykładowy tekst: "MSFT EPS for the last quarter is 3.23 USD despite the estimation of 3.11 USD. In the next quarter EPS is expected to reach 3.22 USD. ..."
                pattern = r"EPS for the last quarter is\s*([\d\.]+)\s*USD\s*.*?estimation of\s*([\d\.]+)\s*USD\s*.*?next quarter EPS is expected to reach\s*([\d\.]+)\s*USD"
                res = search_pattern(pattern, answer_text_clean)
                if res:
                    eps_last_quarter, eps_last_quarter_estimation, eps_next_quarter_expected = res

            # Parsowanie revenue forecast - zapisujemy cały tekst odpowiedzi
            elif "What is AMD revenue forecast for the next quarter" in question:
                next_quarter_revenue = answer_text_clean.strip()

            # Parsowanie price target
            elif "What is AMD price target" in question:
                # Przykładowy tekst: "According to analysts, MSFT price target is 508.48 USD with a max estimate of 650.00 USD and a min estimate of 425.00 USD. ..."
                pattern = r"price target is\s*([\d\.]+)\s*USD\s*with a max estimate of\s*([\d\.]+)\s*USD\s*and a min estimate of\s*([\d\.]+)\s*USD"
                res = search_pattern(pattern, answer_text_clean)
                if res:
                    price_target, max_estimate, min_estimate = res

            # Parsowanie liczby analityków
            elif "What is AMD analyst rating" in question:
                # Przykładowy tekst: "We've gathered opinions of 60 analysts rating MSFT stock in the past 3 months. ..."
                pattern = r"opinions of\s*(\d+)\s*analysts"
                res = search_pattern(pattern, answer_text_clean)
                if res:
                    num_analysts = res[0]

# Utworzenie słownika z wyciągniętymi danymi
data_dict = {
    "Price tv": price_value,
    "EPS last quarter tv": eps_last_quarter,
    "EPS last quarter estimation tv": eps_last_quarter_estimation,
    "EPS next quarter expected tv": eps_next_quarter_expected,
    "price target tv": price_target,
    "max estimate tv": max_estimate,
    "min estimate tv": min_estimate,
    "number of analysts tv": num_analysts,
    "next quarter expected revenue tv": next_quarter_revenue,
    "Beta (1Y) tv": beta,
    "Shares float tv": shares_text
}

final_data_dict = data_dict | sentiment_data | final_flattened_data

# Utworzenie DataFrame
df = pd.DataFrame([final_data_dict])


def extract_revenue(text):
    # Usuń NBSP (\xa0) oraz kilka znaków kierunkowych Unicode (np. \u200E, \u202A, \u202C) oraz narrow no-break space (\u202F)
    cleaned = text.replace(u'\xa0', ' ')
    cleaned = re.sub(r'[\u200E\u202A\u202C\u202F]', '', cleaned)
    # Wyrażenie regularne szuka słowa "reach", a następnie non-greedy grupy zawierającej liczbę z jednostką (B, K lub M)
    pattern = r"reach.*?([\d\.,]+\s*[BKM])"
    m = re.search(pattern, cleaned, re.IGNORECASE | re.DOTALL)
    return m.group(1).strip() if m else None

# Zastępujemy wartość w kolumnie "next quarter expected revenue" bezpośrednio wyekstrahowaną wartością
df["next quarter expected revenue tv"] = df["next quarter expected revenue tv"].apply(extract_revenue)

Beta (1Y): 2.51
Shares cleaned: 7.53BUSD
Przetwarzam zakładkę: 1D - 1 day (sufiks: 1d)
Przetwarzam zakładkę: 1W - 1 week (sufiks: 1w)
Przetwarzam zakładkę: 1M - 1 month (sufiks: 1M)

Sentiment data: {'Strong buy tv': 0, 'Buy tv': 0, 'Hold tv': 0, 'Sell tv': 0, 'Strong sell tv': 0}


In [201]:
df

,Price tv,EPS last quarter tv,EPS last quarter estimation tv,EPS next quarter expected tv,price target tv,max estimate tv,min estimate tv,number of analysts tv,next quarter expected revenue tv,Beta (1Y) tv,...,Exponential Moving Average (200) - Value tv 1M,Exponential Moving Average (200) - Action tv 1M,Simple Moving Average (200) - Value tv 1M,Simple Moving Average (200) - Action tv 1M,"Ichimoku Base Line (9, 26, 52, 26) - Value tv 1M","Ichimoku Base Line (9, 26, 52, 26) - Action tv 1M",Volume Weighted Moving Average (20) - Value tv 1M,Volume Weighted Moving Average (20) - Action tv 1M,Hull Moving Average (9) - Value tv 1M,Hull Moving Average (9) - Action tv 1M
0,,0.92,0.92,1.08,169.00,250.00,110.00,56,7.53B,2.51,...,57.98,Buy,38.55,Buy,143.68,Neutral,143.40,Sell,110.71,Buy


In [197]:
cols = list(df.columns)
print(len(cols))
cols

172


['Price tv',
 'EPS last quarter tv',
 'EPS last quarter estimation tv',
 'EPS next quarter expected tv',
 'price target tv',
 'max estimate tv',
 'min estimate tv',
 'number of analysts tv',
 'next quarter expected revenue tv',
 'Beta (1Y) tv',
 'Shares float tv',
 'Strong buy tv',
 'Buy tv',
 'Hold tv',
 'Sell tv',
 'Strong sell tv',
 'Relative Strength Index (14) - Value tv 1d',
 'Relative Strength Index (14) - Action tv 1d',
 'Stochastic %K (14, 3, 3) - Value tv 1d',
 'Stochastic %K (14, 3, 3) - Action tv 1d',
 'Commodity Channel Index (20) - Value tv 1d',
 'Commodity Channel Index (20) - Action tv 1d',
 'Average Directional Index (14) - Value tv 1d',
 'Average Directional Index (14) - Action tv 1d',
 'Awesome Oscillator - Value tv 1d',
 'Awesome Oscillator - Action tv 1d',
 'Momentum (10) - Value tv 1d',
 'Momentum (10) - Action tv 1d',
 'MACD Level (12, 26) - Value tv 1d',
 'MACD Level (12, 26) - Action tv 1d',
 'Stochastic RSI Fast (3, 3, 14, 14) - Value tv 1d',
 'Stochastic RSI 

In [189]:
final_flattened_data

{'Relative Strength Index (14) - Value 1d': '40.77',
 'Relative Strength Index (14) - Action 1d': 'Neutral',
 'Stochastic %K (14, 3, 3) - Value 1d': '35.69',
 'Stochastic %K (14, 3, 3) - Action 1d': 'Neutral',
 'Commodity Channel Index (20) - Value 1d': '−67.99',
 'Commodity Channel Index (20) - Action 1d': 'Neutral',
 'Average Directional Index (14) - Value 1d': '21.30',
 'Average Directional Index (14) - Action 1d': 'Neutral',
 'Awesome Oscillator - Value 1d': '−5.71',
 'Awesome Oscillator - Action 1d': 'Neutral',
 'Momentum (10) - Value 1d': '−2.49',
 'Momentum (10) - Action 1d': 'Sell',
 'MACD Level (12, 26) - Value 1d': '−2.82',
 'MACD Level (12, 26) - Action 1d': 'Buy',
 'Stochastic RSI Fast (3, 3, 14, 14) - Value 1d': '54.32',
 'Stochastic RSI Fast (3, 3, 14, 14) - Action 1d': 'Neutral',
 'Williams Percent Range (14) - Value 1d': '−75.49',
 'Williams Percent Range (14) - Action 1d': 'Neutral',
 'Bull Bear Power - Value 1d': '−3.17',
 'Bull Bear Power - Action 1d': 'Sell',
 'Ulti

In [158]:
import os
data = "/Users/michal/PycharmProjects/Stock Scraper/tradingviewlinks.csv"
datatickers = pd.read_csv(data, delimiter=";")
datatickers

# core = "https://www.tradingview.com/symbols/"
# tail = "/forecast/"
# 
# datatickers["tradingview link"] = [
#     core + exchange + "-" + symbol + tail 
#     for exchange, symbol in zip(datatickers["Exchange"], datatickers["Symbol"])
# ]
# 
# out = "/Users/michal/PycharmProjects/Stock Scraper/tradingviewlinks"
# file_exists = os.path.isfile(out)
# datatickers.to_csv(
#     out,
#     index=False,
#     mode='w',  # 'a' dla append (dopisywanie), 'w' dla write (pisanie od nowa)
#     header=True,     #not file_exists,  # Zapisz nagłówek tylko jeśli plik nie istnieje
#     sep=';'
# )
    
# datatickers

,Exchange,Symbol,tradingview link
0,NASDAQ,AAPL,https://www.tradingview.com/symbols/NASDAQ-AAPL/
1,NASDAQ,NVDA,https://www.tradingview.com/symbols/NASDAQ-NVDA/
2,NASDAQ,MSFT,https://www.tradingview.com/symbols/NASDAQ-MSFT/
3,NASDAQ,AMZN,https://www.tradingview.com/symbols/NASDAQ-AMZN/
4,NASDAQ,GOOGL,https://www.tradingview.com/symbols/NASDAQ-GOOGL/
...,...,...,...
500,NYSE,FMC,https://www.tradingview.com/symbols/NYSE-FMC/
501,NYSE,AMTM,https://www.tradingview.com/symbols/NYSE-AMTM/
502,NYSE,TSM,https://www.tradingview.com/symbols/NYSE-TSM/
503,NASDAQ,MGX,https://www.tradingview.com/symbols/NASDAQ-MGX/


In [159]:
datatickers

,Exchange,Symbol,tradingview link
0,NASDAQ,AAPL,https://www.tradingview.com/symbols/NASDAQ-AAPL/
1,NASDAQ,NVDA,https://www.tradingview.com/symbols/NASDAQ-NVDA/
2,NASDAQ,MSFT,https://www.tradingview.com/symbols/NASDAQ-MSFT/
3,NASDAQ,AMZN,https://www.tradingview.com/symbols/NASDAQ-AMZN/
4,NASDAQ,GOOGL,https://www.tradingview.com/symbols/NASDAQ-GOOGL/
...,...,...,...
500,NYSE,FMC,https://www.tradingview.com/symbols/NYSE-FMC/
501,NYSE,AMTM,https://www.tradingview.com/symbols/NYSE-AMTM/
502,NYSE,TSM,https://www.tradingview.com/symbols/NYSE-TSM/
503,NASDAQ,MGX,https://www.tradingview.com/symbols/NASDAQ-MGX/


In [156]:
for link in datatickers["tradingview link"]:
    link = link.rstrip("forecast/")
    print(link)

https://www.tradingview.com/symbols/NASDAQ-AAPL
https://www.tradingview.com/symbols/NASDAQ-NVDA
https://www.tradingview.com/symbols/NASDAQ-MSFT
https://www.tradingview.com/symbols/NASDAQ-AMZN
https://www.tradingview.com/symbols/NASDAQ-GOOGL
https://www.tradingview.com/symbols/NASDAQ-GOOG
https://www.tradingview.com/symbols/NASDAQ-META
https://www.tradingview.com/symbols/NASDAQ-TSLA
https://www.tradingview.com/symbols/NASDAQ-AVGO
https://www.tradingview.com/symbols/NYSE-BRK-B
https://www.tradingview.com/symbols/NYSE-WMT
https://www.tradingview.com/symbols/NYSE-LLY
https://www.tradingview.com/symbols/NYSE-JPM
https://www.tradingview.com/symbols/NYSE-V
https://www.tradingview.com/symbols/NYSE-MA
https://www.tradingview.com/symbols/NYSE-ORCL
https://www.tradingview.com/symbols/NYSE-XOM
https://www.tradingview.com/symbols/NYSE-UNH
https://www.tradingview.com/symbols/NASDAQ-COST
https://www.tradingview.com/symbols/NYSE-PG
https://www.tradingview.com/symbols/NYSE-HD
https://www.tradingview.co

In [162]:
# Ścieżka do pliku chromedriver (zmień na właściwą lokalizację)
driver_path = "/Users/michal/Downloads/chromedriver-mac-x64/chromedriver"

# Inicjalizacja sterownika Chrome
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Ustawienie WebDriverWait na 30 sekund
wait = WebDriverWait(driver, 30)

# Ładujemy stronę główną dla AMD (bez /forecast/)
url_main = "https://www.tradingview.com/symbols/NASDAQ-AMD/technicals/"
driver.get(url_main)

# Zakładamy, że driver jest już zainicjalizowany i strona zawiera fragment HTML poniżej
# Znajdujemy kontener wszystkich tabel
tables_wrapper = driver.find_element(By.CSS_SELECTOR, "div.tablesWrapper-kg4MJrFB.tabletVertical-kg4MJrFB")

# Znajdujemy wszystkie kontenery tabel – każdy taki kontener zazwyczaj zawiera tytuł i jedną tabelę
table_containers = tables_wrapper.find_elements(By.CSS_SELECTOR, "div.container-hvDpy38G")

extracted_data = {}

for container in table_containers:
    # Pobieramy tytuł sekcji (np. Oscillators, Moving Averages)
    try:
        title_element = container.find_element(By.CSS_SELECTOR, "div.title-hvDpy38G a.link-hvDpy38G")
        section_title = title_element.text.strip()
    except Exception as e:
        section_title = "Brak tytułu"
    
    # Pobieramy tabelę wewnątrz kontenera
    try:
        table = container.find_element(By.CSS_SELECTOR, "table.table-hvDpy38G")
    except Exception as e:
        continue  # jeśli nie ma tabeli, pomijamy ten kontener

    # Pobieramy wszystkie wiersze, które nie są nagłówkiem (zwykle mają klasę "row-hvDpy38G")
    rows = table.find_elements(By.CSS_SELECTOR, "tr.row-hvDpy38G")
    rows_data = []
    for row in rows:
        # Każdy wiersz powinien mieć co najmniej trzy komórki: Name, Value, Action
        cells = row.find_elements(By.CSS_SELECTOR, "td.cell-hvDpy38G")
        if len(cells) >= 3:
            name = cells[0].text.strip()
            value = cells[1].text.strip()
            action = cells[2].text.strip()
            rows_data.append({
                "Name": name,
                "Value": value,
                "Action": action
            })
    extracted_data[section_title] = rows_data

# Wypisujemy wynik – słownik z danymi z poszczególnych sekcji
print("Dane z tabel:", extracted_data)

flattened_data = {}

# Iterujemy po każdej kategorii i po każdym elemencie w liście
for category, items in extracted_data.items():
    for item in items:
        # Pobieramy nazwę, wartość oraz akcję
        name = item.get("Name", "").strip()
        value = item.get("Value", "").strip()
        action = item.get("Action", "").strip()
        # Tworzymy nowe klucze np. "Relative Strength Index (14) - Value" oraz "Relative Strength Index (14) - Action"
        flattened_data[f"{name} - Value"] = value
        flattened_data[f"{name} - Action"] = action

# Tworzymy DataFrame z pojedynczym wierszem
df_flat = pd.DataFrame([flattened_data])
print(df_flat)

Dane z tabel: {'Oscillators': [{'Name': 'Relative Strength Index (14)', 'Value': '40.77', 'Action': 'Neutral'}, {'Name': 'Stochastic %K (14, 3, 3)', 'Value': '35.69', 'Action': 'Neutral'}, {'Name': 'Commodity Channel Index (20)', 'Value': '−67.99', 'Action': 'Neutral'}, {'Name': 'Average Directional Index (14)', 'Value': '21.30', 'Action': 'Neutral'}, {'Name': 'Awesome Oscillator', 'Value': '−5.71', 'Action': 'Neutral'}, {'Name': 'Momentum (10)', 'Value': '−2.49', 'Action': 'Sell'}, {'Name': 'MACD Level (12, 26)', 'Value': '−2.82', 'Action': 'Buy'}, {'Name': 'Stochastic RSI Fast (3, 3, 14, 14)', 'Value': '54.32', 'Action': 'Neutral'}, {'Name': 'Williams Percent Range (14)', 'Value': '−75.49', 'Action': 'Neutral'}, {'Name': 'Bull Bear Power', 'Value': '−3.17', 'Action': 'Sell'}, {'Name': 'Ultimate Oscillator (7, 14, 28)', 'Value': '41.66', 'Action': 'Neutral'}], 'Moving Averages': [{'Name': 'Exponential Moving Average (10)', 'Value': '118.48', 'Action': 'Sell'}, {'Name': 'Simple Moving 

In [167]:
flattened_data

{'Relative Strength Index (14) - Value': '40.77',
 'Relative Strength Index (14) - Action': 'Neutral',
 'Stochastic %K (14, 3, 3) - Value': '35.69',
 'Stochastic %K (14, 3, 3) - Action': 'Neutral',
 'Commodity Channel Index (20) - Value': '−67.99',
 'Commodity Channel Index (20) - Action': 'Neutral',
 'Average Directional Index (14) - Value': '21.30',
 'Average Directional Index (14) - Action': 'Neutral',
 'Awesome Oscillator - Value': '−5.71',
 'Awesome Oscillator - Action': 'Neutral',
 'Momentum (10) - Value': '−2.49',
 'Momentum (10) - Action': 'Sell',
 'MACD Level (12, 26) - Value': '−2.82',
 'MACD Level (12, 26) - Action': 'Buy',
 'Stochastic RSI Fast (3, 3, 14, 14) - Value': '54.32',
 'Stochastic RSI Fast (3, 3, 14, 14) - Action': 'Neutral',
 'Williams Percent Range (14) - Value': '−75.49',
 'Williams Percent Range (14) - Action': 'Neutral',
 'Bull Bear Power - Value': '−3.17',
 'Bull Bear Power - Action': 'Sell',
 'Ultimate Oscillator (7, 14, 28) - Value': '41.66',
 'Ultimate Os